In [34]:
import sqlite3

# Connect to the SQLite database
connection = sqlite3.connect('registry.db')
cursor = connection.cursor()

# SQL statements to drop tables if they exist
drop_tables_sql = [
    "DROP TABLE IF EXISTS value;",
    "DROP TABLE IF EXISTS statement;",
    "DROP TABLE IF EXISTS property;",
    "DROP TABLE IF EXISTS record;",
    "DROP TABLE IF EXISTS registry;"
]

# Execute each DROP TABLE SQL statement
for sql in drop_tables_sql:
    cursor.execute(sql)

# SQL statements to create new tables
create_tables_sql = [
    """
    CREATE TABLE IF NOT EXISTS registry (
        id INTEGER PRIMARY KEY AUTOINCREMENT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS record (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        registry_id INTEGER,
        identifier TEXT,
        FOREIGN KEY (registry_id) REFERENCES registry(id)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS property (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        type_name TEXT,
        property_name TEXT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS statement (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        record_id INTEGER,
        property_id INTEGER,
        effective_from DATETIME,
        created DATETIME,
        FOREIGN KEY (record_id) REFERENCES record(id),
        FOREIGN KEY (property_id) REFERENCES property(id)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS value (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        statement_id INTEGER,
        value TEXT,
        FOREIGN KEY (statement_id) REFERENCES statement(id)
    );
    """
]

# Execute each CREATE TABLE SQL statement
for sql in create_tables_sql:
    cursor.execute(sql)

# Commit the changes and close the database connection
connection.commit()
connection.close()


In [35]:
import sqlite3

connection = sqlite3.connect('registry.db')
cursor = connection.cursor()

dummy_data_statements = [
    "INSERT INTO registry (id) VALUES (1), (2);",
    "INSERT INTO record (registry_id, identifier) VALUES (1, '90110800000');",
    """
    INSERT INTO property (id, type_name, property_name)
    VALUES 
    (1, 'Person', 'firstName'),
    (2, 'Person', 'citizenship'),
    (3, 'Residency', 'residence');
    """,
    """
    INSERT INTO statement (record_id, property_id, effective_from, created)
    VALUES
    (1, 1, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO value (statement_id, value)
    VALUES
    (1, 'John')
    """,
    """
    INSERT INTO statement (record_id, property_id, effective_from, created)
    VALUES 
    (1, 2, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO value (statement_id, value)
    VALUES
    (2, '{"isoCode": "BEL", "nisCode": "150"}')
    """,
    """
    INSERT INTO statement (record_id, property_id, effective_from, created)
    VALUES 
    (1, 3, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO value (statement_id, value)
    VALUES 
    (3, '{"address": {"bestIdentifier": {"id": "https://data.vlaanderen.be/id/adres/3706808", "namespace": "https://data.vlaanderen.be/id/adres", "objectId": "3706808", "versionId": "2023-11-01T15:46:20+01:00"}}}');
    """,
    """
    INSERT INTO statement (record_id, property_id, effective_from, created)
    VALUES
    (1, 1, '2021-01-01 10:00:00', '2021-01-01 10:00:00');
    """,
    """
    INSERT INTO value (statement_id, value)
    VALUES
    (4, 'Tom')
    """
]

for statement in dummy_data_statements:
    cursor.execute(statement)

connection.commit()
connection.close()

In [36]:
import json
import pandas as pd
import sqlite3

connection = sqlite3.connect('registry.db')
cursor = connection.cursor()

query = """
SELECT s.id AS statement, r.identifier, p.type_name, p.property_name, v.value, s.effective_from, s.created 
FROM statement s
JOIN record r ON s.record_id = r.id
JOIN property p ON s.property_id = p.id
JOIN value v ON v.statement_id = s.id;
"""
statement_df = pd.read_sql_query(query, connection)

connection.close()

statement_df

# Convert the DataFrame into a JSON string
json_str = statement_df.to_json(orient="records", lines=False)

json_data = json.loads(json_str)

formatted_json = json.dumps(json_data, indent=4)

print(formatted_json)

[
    {
        "statement": 1,
        "identifier": "90110800000",
        "type_name": "Person",
        "property_name": "firstName",
        "value": "John",
        "effective_from": "2020-01-01 10:00:00",
        "created": "2020-01-01 10:00:00"
    },
    {
        "statement": 2,
        "identifier": "90110800000",
        "type_name": "Person",
        "property_name": "citizenship",
        "value": "{\"isoCode\": \"BEL\", \"nisCode\": \"150\"}",
        "effective_from": "2020-01-01 10:00:00",
        "created": "2020-01-01 10:00:00"
    },
    {
        "statement": 3,
        "identifier": "90110800000",
        "type_name": "Residency",
        "property_name": "residence",
        "value": "{\"address\": {\"bestIdentifier\": {\"id\": \"https://data.vlaanderen.be/id/adres/3706808\", \"namespace\": \"https://data.vlaanderen.be/id/adres\", \"objectId\": \"3706808\", \"versionId\": \"2023-11-01T15:46:20+01:00\"}}}",
        "effective_from": "2020-01-01 10:00:00",
        

In [37]:
import sqlite3

connection = sqlite3.connect('registry.db')
cursor = connection.cursor()

# Return current state of record
query = """
SELECT 
    r.identifier, p.type_name, p.property_name, v.value, s.effective_from, MAX(s.created) AS last_modified
FROM 
    statement s
JOIN record r ON s.record_id = r.id
JOIN property p ON s.property_id = p.id
JOIN value v ON v.statement_id = s.id
WHERE
    s.record_id = 1
GROUP BY 
    p.id;
"""

statement_df = pd.read_sql_query(query, connection)

connection.close()

statement_df

,identifier,type_name,property_name,value,effective_from,last_modified
0,90110800000,Person,firstName,Tom,2021-01-01 10:00:00,2021-01-01 10:00:00
1,90110800000,Person,citizenship,"{""isoCode"": ""BEL"", ""nisCode"": ""150""}",2020-01-01 10:00:00,2020-01-01 10:00:00
2,90110800000,Residency,residence,"{""address"": {""bestIdentifier"": {""id"": ""https:/...",2020-01-01 10:00:00,2020-01-01 10:00:00
